In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
pd.set_option("display.max_rows", None, "display.max_columns", None)
#1. time
#2. dow jones (value and plus minus)
#3. open interest (price, oi, change, implied volitality, trade price) 
#4. Current nifty price 


In [3]:
#1. time
from datetime import datetime
now = datetime.now().time()
time = now 
print(time)

04:47:56.461023


In [4]:
#CALL DATA 
url = "https://finance.yahoo.com/quote/DIA/options?date=1588896000" #url
response = requests.get(url)

soup = BeautifulSoup(response.text,'html.parser')
table = soup.find_all('table',{'class' : 'calls W(100%) Pos(r) Bd(0) Pt(0) list-options'})
call = pd.read_html(str(table))
call= call[0]

#PUT DATA 

response = requests.get(url)

soup = BeautifulSoup(response.text,'html.parser')
table = soup.find_all('table')[1] 
puts = pd.read_html(str(table))
put= puts[0]
print(put.columns)

Index(['Contract Name', 'Last Trade Date', 'Strike', 'Last Price', 'Bid',
       'Ask', 'Change', '% Change', 'Volume', 'Open Interest',
       'Implied Volatility'],
      dtype='object')


In [5]:
#contract name (call)
#Last Trade Date (call)
#Open Interest(call)
#Volume(call)
#Strike(call)
#Open Interest(put)
#Volume(put)


In [6]:
valid_strike =[]
for i in call['Strike']:
    if (i<=260 and i>=220):
        valid_strike.append(i)

contract_name =[]
Last_Trade_Date = []
Open_Interest_call = []
Volume_call = []
Strike = []
Open_Interest_put = []
Volume_put = []

for i in range(len(call)):
    if(call['Strike'][i] in valid_strike):
        contract_name.append(call['Contract Name'][i])
        Last_Trade_Date.append(call['Last Trade Date'][i])
        Open_Interest_call.append(float(call['Open Interest'][i]))
        Volume_call.append(float(call['Volume'][i]))
        Strike.append(float(call['Strike'][i]))
        
for i in range(len(put)):
    if(put['Strike'][i] in valid_strike):
        Volume_put.append(float(put['Volume'][i]))
        Open_Interest_put.append(float(put['Open Interest'][i]))



In [7]:
data = pd.DataFrame(contract_name,columns={"Contract_name"})
#data['Last_Trade_Date'] = Last_Trade_Date
data['Open_Interest_call'] = Open_Interest_call
data['Volume_call'] = Volume_call
data['Strike'] = Strike
data['Volume_put'] = Volume_put
data['Open_Interest_put'] = Open_Interest_put


In [21]:
#2. dow jones (value and plus minus)
url = "https://www.cnbc.com/quotes/?symbol=@DJ.1" #url
response = requests.get(url)

soup = BeautifulSoup(response.text,'html.parser')

spans = soup.findAll('span', {'class' : 'last original'})
for span in spans:
    dow = span.text
    break
    
    
##################################################################################
put_total = 0
call_total = 0
for i in range(len(data["Open_Interest_call"])):
    call_total = call_total + data["Open_Interest_call"][i] 

for i in range(len(data["Open_Interest_put"])):
    put_total = put_total + data["Open_Interest_put"][i] 
##################################################################################
#Point of maximum pain:
options = []
for i in range(len(data)):
    options.append(data['Open_Interest_call'][i]+data['Open_Interest_put'][i])

max_pain = 0
for i in range(len(options)):
    if(options[i]==max(options)):
        max_pain1 = data['Strike'][i]
options.remove(max(options))

max_pain2 = 0
for i in range(len(options)):
    if(options[i]==max(options)):
        max_pain2 = data['Strike'][i]

options.remove(max(options))

max_pain3 = 0
for i in range(len(options)):
    if(options[i]==max(options)):
        max_pain3 = data['Strike'][i]
#2nd max point
##################################################################################
print("\t\t\t\t\tDOW FUTURE =",float(dow)/100)
print("\t\t\t\t\tMaximum pain point 1 =",max_pain1)
print("\t\t\t\t\tMaximum pain point 2 =",max_pain2)
print("\t\t\t\t\tMaximum pain point 3 =",max_pain3)
print("\t\t\t\t\tPUTS/CALLS =",put_total/call_total)

data.style.format({"Open_Interest_call": "{:20,.0f}", 
                            "Volume_call": "{:20,.0f}", 
                          "Volume_put": "{:20,.0f}",
                          "Open_Interest_put":"{:20,.0f}"})\
                 .hide_index()\
                 .bar(subset=["Open_Interest_call",], color='lightgreen')\
                 .bar(subset=["Open_Interest_put"], color='#ee1f5f')




					DOW FUTURE = 235.84
					Maximum pain point 1 = 235.0
					Maximum pain point 2 = 242.5
					Maximum pain point 3 = 237.5
					PUTS/CALLS = 1.3425052410901468


Contract_name,Open_Interest_call,Volume_call,Strike,Volume_put,Open_Interest_put
DIA200508C00220000,55,5,220,212,568
DIA200508C00221000,1,1,221,18,42
DIA200508C00222000,5,1,222,59,49
DIA200508C00222500,4,1,222.5,6,26
DIA200508C00224000,5,5,224,16,34
DIA200508C00225000,34,2,225,143,206
DIA200508C00227500,68,2,227.5,89,142
DIA200508C00230000,75,38,230,781,540
DIA200508C00232500,56,9,232.5,578,286
DIA200508C00235000,160,53,235,345,"1,652"
